## Représentation sous forme de diagramme de Vornoi des zones nécessitant un aménagement d'arceaux à vélo sur Brest

In [1]:
# Import necessary packages
import requests  # For making HTTP requests
import pandas as pd  # For data manipulation and analysis
import overpy  # For accessing OpenStreetMap data
import numpy as np  # For numerical operations
import scipy.spatial as spatial  # For spatial operations
import warnings # For disabling future warnings
import folium # Plot the points on the map as red circles

In [2]:
# Suppressing the FutureWarning related to is_categorical_dtype
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Make an HTTP GET request to fetch data from a specified URL
    resp = requests.get('https://vigilo.bapav.org/get_issues.php?scope=29_brest&format=json&c=5') # vigilo database of BaPaVe
    
    # Convert the JSON response to a DataFrame
    df = pd.DataFrame(resp.json())
    
    # Filter out rows where the 'comment' column is empty
    df = df[df.comment != '']
    
    # Convert the 'time' column to datetime format, assuming it's in seconds since epoch
    df.time = pd.to_datetime(df.time, unit='s')
    
    # Add a new column 'julian_day' representing the day of the year
    df['julian_day'] = df.time.apply(lambda x: x.timetuple().tm_yday)
    
    # Filter out data before January 1, 2020
    df = df[df.time.apply(lambda x: x.year) > 2019]
    
    # Sort the DataFrame by the 'time' column in ascending order
    df = df.sort_values(by='time', ascending=True)
    
    # Reset the index after sorting
    df = df.reset_index(drop=True)

In [3]:
# Définir les coordonnées de la zone d'intérêt
box = (48.377, -4.556, 
       48.416, -4.441)

# Créer une requête Overpass API pour récupérer les arceaux à vélos ou les parkings à vélos
api = overpy.Overpass()

# Requête pour les arceaux à vélos
query = f"""
    node["amenity"="bicycle_parking"]({box[0]},{box[1]},{box[2]},{box[3]});
    out;
"""

# Exécuter la requête
result = api.query(query)

# Récupérer les coordonnées des arceaux à vélos
arceaux_coords = [(node.lat, node.lon) for node in result.nodes]

# Créer un DataFrame à partir de arceaux_coords
df_coords = pd.DataFrame(arceaux_coords, columns=['Latitude', 'Longitude'])

In [4]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """Reconstruct infinite Voronoi regions in a
    2D diagram to finite regions.
    
    Parameters:
    - vor: scipy.spatial.Voronoi object
           Voronoi diagram computed using the scipy library.
    - radius: float, optional
              Radius of the bounding circle around the original points.
              If not provided, it is set to the maximum point-to-point
              distance in the Voronoi diagram.
              
    Returns:
    - new_regions: list
                   List of finite polygons representing Voronoi regions.
    - new_vertices: numpy array
                    Array containing the coordinates of the new vertices
                    added during reconstruction.
    
    Source:
    [https://stackoverflow.com/a/20678647/1595060](https://stackoverflow.com/a/20678647/1595060)
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")
    
    new_regions = []  # List to store finite polygons
    new_vertices = vor.vertices.tolist()  # List to store new vertices
    
    center = vor.points.mean(axis=0)  # Compute the center of the points
    
    # If radius is not provided, set it to the maximum point-to-point distance
    if radius is None:
        radius = vor.points.ptp().max()
    
    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))
    
    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        
        if all(v >= 0 for v in vertices):
            # Finite region: already in the new_regions list
            new_regions.append(vertices)
            continue
        
        # Reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # Finite ridge: already in the region
                continue
            
            # Compute the missing endpoint of an infinite ridge
            t = vor.points[p2] - vor.points[p1]  # Tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # Normal
            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius
            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
        
        # Sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]
        new_regions.append(new_region.tolist())
    
    return new_regions, np.asarray(new_vertices)

In [5]:
# Initialize an empty list to store coordinate tuples
coos = []

# Iterate over the rows of the DataFrame to extract coordinates
for i in range(len(df_coords.Latitude)):
    # Extract latitude and longitude values from the DataFrame
    x = float(df_coords.Latitude.values[i])
    y = float(df_coords.Longitude.values[i])

    # Append the coordinate tuple (latitude, longitude) to the list
    coos.append((x, y))

# Iterate over the rows of the DataFrame to extract coordinates
for i in range(len(df.coordinates_lat)):
    # Extract latitude and longitude values from the DataFrame
    x = float(df.coordinates_lat.values[i])
    y = float(df.coordinates_lon.values[i])

    # Append the coordinate tuple (latitude, longitude) to the list
    coos.append((x, y))


In [6]:
# Compute the Voronoi diagram for the given coordinates
vor = spatial.Voronoi(coos)

# Reconstruct finite polygons from infinite Voronoi regions
regions, vertices = voronoi_finite_polygons_2d(vor)

In [7]:
# Créer une carte centrée sur Brest
m = folium.Map(location=[48.3992389, -4.4799506], zoom_start=14)

# Add the Voronoi regions to the map
for region in regions:
    polygon = folium.Polygon(locations=vertices[region], color='blue', fill=True, fill_color='blue', fill_opacity=0.1)
    m.add_child(polygon)

# Ajouter les points individuels à la carte
for i in range(len(df_coords)):
    lat = float(df_coords['Latitude'].iloc[i])
    lon = float(df_coords['Longitude'].iloc[i])
    folium.CircleMarker(location=[lat, lon], radius=5, color='red', fill=True, fill_color='red').add_to(m)
for i in range(len(df)):
    lat = float(df['coordinates_lat'].iloc[i])
    lon = float(df['coordinates_lon'].iloc[i])
    folium.CircleMarker(location=[lat, lon], radius=5, color='green', fill=True, fill_color='green').add_to(m)

# Afficher la carte
m